# Step One: Scraping

Initial scraping using Notebook, BeautifulSoup, Pandas & Requests/Splinter

In [2]:
#import things
import pandas as pd
from bs4 import BeautifulSoup
from splinter import Browser
import requests

## Nasa Mars News

Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.

In [3]:
#define a variable to store the variable for the website

url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'

In [4]:
# identify location of chromedriver and store it as a variable
driverPath = !which chromedriver

# Setup configuration variables to enable Splinter to interact with browser
executable_path = {'executable_path': driverPath[0]}
browser = Browser('chrome', **executable_path, headless=False)

In [5]:
#inform the browser to visit the page
browser.visit(url)

In [6]:
# Splinter can capture a page's underlying html and use pass it to BeautifulSoup to allow us to scrape the content
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

# Using BS, we can execute standard functions to capture the page's content
title = soup.find_all('div', class_='content_title')[1].text
p_words = soup.find_all('div', class_='article_teaser_body')[0].text 

print(title)
print('------')
print(p_words)

Sensors on Mars 2020 Spacecraft Answer Long-Distance Call From Earth
------
Instruments tailored to collect data during the descent of NASA's next rover through the Red Planet's atmosphere have been checked in flight.


## JPL Mars Space Images - Featured Images

In [7]:
#define the variable to store the url for landing image

image_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

browser.visit(image_url)

In [8]:
# Splinter can capture a page's underlying html and use pass it to BeautifulSoup to allow us to scrape the content
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

# print(soup.prettify())


In [9]:
#try to get all of the html related to images
images = soup.find_all('a', class_='fancybox')
images

large_image = []
for image in images:
    photo = image['data-fancybox-href']
    large_image.append(photo)
    
large_image

header_img = 'https://www.jpl.nasa.gov' + photo
header_img


'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA24112_hires.jpg'

## Mars Facts

Visit Mars Facts webpage and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.

Use Pandas to convert the data to a HTML table string

In [10]:
#define a variable to to the url for mars facts

url = 'https://space-facts.com/mars/'

In [11]:
#read html page with pandas as requested

facts = pd.read_html(url)
len(facts)

df = facts[2]
df.keys()

df.columns = ['', 'Mars']
df

,,Mars
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [52]:
html_table = df.to_html()
print(html_table)

df.to_html('table.html')

!open table.html

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th></th>
      <th>Mars</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Equatorial Diameter:</td>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Polar Diameter:</td>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Mass:</td>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Moons:</td>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Orbit Distance:</td>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <th>5</th>
      <td>Orbit Period:</td>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>6</th>
      <td>Surface Temperature:</td>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <th>7</th>
      <td>First Record:</td>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>8</th>
      <td>Record

## Mars Hemispheres

Visit the USGS Astrogeology site to obtain hi-res images for each of Mar's hemispheres

<ul> You will need to click each link to the hemispheres to find the image url for the full-res image </ul>

<ul> Save both image url string for the full res hemisphere image, and the Hemisphere title containing the hemisphere name. Use Python dictioniary to store the data using the keys img_url and title</ul>

<ul> Append the dictionary with the image url string and hemisphere time to a list. This list will contain one dictionary for each hemisphere. </ul>

In [47]:
#get the url 
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

browser.visit(url)

#use BS to bring in the browser
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

#create the list
mars_urls = []

#retrieve all elements that contain the information needed 
results = soup.find('div', class_='result-list')
each_hemisphere = results.find_all('div', class_='item')

In [49]:
#create a for loop to get all the information
for i in each_hemisphere:
    title = i.find('h3').text
    title = title.replace("Enchanced", "")
    each_link = i.find('a')['href']
    img_url = 'https://astrogeology.usgs.gov/' + each_link
#     get the link to visit each page and get the image 
    browser.visit(img_url)
    html = browser.html
    soup = BeautifulSoup(html,'html.parser')
    each_pg = soup.find('div', class_='downloads')
    image = each_pg.find('a')['href']
    mars_urls.append({'title': title, 'img_url': image})

In [50]:
print(mars_urls)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]
